In [1]:
!pip install pyspark

In [2]:
import pandas as pd

from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F

from google.colab import drive
drive.mount('/content/drive')

INPUT_DATA_PATH  = "/content/drive/MyDrive/basic_features"

Mounted at /content/drive


In [3]:
spark = SparkSession.builder.getOrCreate()

# Import Data

In [4]:
# Preprocessed Walmart Dataset
df = spark.read.parquet(INPUT_DATA_PATH)

# 1 - Calcul du Chiffre D'affaire

In [5]:
df = (
    df
    .withColumn("ca", F.col("sales") * F.col("avg_price"))
    .fillna(0, subset = ["ca"])
)

# 2 - Encodage Cyclique des Mois

L'encodage cyclique est une technique utile pour représenter des variables temporelles/périodique, comme les mois de l'année. Les mois peuvent être représentés par des valeurs qui varient de 1 à 12, mais en raison de leur nature cyclique, il est préférable de les encoder en utilisant les fonctions sinus et cosinus permettant ainsi aux modèles de machine learning de mieux capter la saisonalité.

Pour calculer les coordonnées \( X \) et \( Y \) de chaque mois sur un cercle unité, les formules sont les suivantes :

- Soit \( m \) le numéro du mois (variant de 1 à 12).
- La coordonnée \( X \) est donnée par :

  $$ X = \cos\left( \frac{2 \pi \times m}{12} \right) $$

- La coordonnée \( Y \) est donnée par :

  $$ Y = \sin\left( \frac{2 \pi \times m }{12} \right) $$




In [6]:
df = (
    df
    .withColumn("cos_month", F.cos((2 * F.pi() * F.col("month")) / 12))
    .withColumn("sin_month", F.sin((2 * F.pi() * F.col("month")) / 12))
)

# 3 - Quotas de marché
Les quotas de marché represente la part des ventes de chaque produit par rapport au total du marché.

Dans notre cas, nous supposerons que chaque catégorie (colonne cat_id) de produit représente le marché correspondant à chaque produit.

Pour calculer la part de marché de chaque produit, on divise les ventes de chaque produit par le total des ventes de sa catégorie.

Il est important de prendre en compte uniquement les observations qui précèdent la date d’observation afin d’éviter tout risque de data leakage.

Par exemple, pour une observation datée du 1er janvier 2020, il est essentiel de réaliser les calculs uniquement sur les observations datées avant cette date. Si les observations ultérieures sont incluses dans le calcul, cela introduirait du data leakage.

Hint : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.rangeBetween.html

In [7]:
w_category = (
    Window.partitionBy("cat_id")
    .orderBy(F.asc("date"))
    .rangeBetween(
        start=Window.unboundedPreceding,
        end=0,
    )
)

w_item = (
    Window.partitionBy("item_id")
    .orderBy(F.asc("date"))
    .rangeBetween(
        start=Window.unboundedPreceding,
        end=0,
    )
)

df = (
    df
    .withColumn(
        "ca_category", F.sum("ca").over(w_category)
    )
    .withColumn(
        "ca_item", F.sum("ca").over(w_item)
    )
    .withColumn("item_market_quota", F.col("ca_item") / F.col("ca_category"))
)

# 3 - Segmentation ABC
La segmentation ABC classe les produits en trois catégories selon leur importance économique :

* A : produits qui génèrent 80 % des ventes. Ce sont les produits les plus stratégiques.
* B : produits qui génèrent 15 % des ventes. Importance modérée.
* C : produits qui génèrent 5 % des ventes.

Dans notre cas, nous allons utiliser le pourcentage du chiffre d’affaires généré par chaque produit par rapport au chiffre d’affaires global de Walmart.

Hint : Les window functions et les when/otherwise sont vos meilleurs amis  !

In [9]:
w_walmart = (
    Window
    .partitionBy()
    .orderBy(F.asc("date"))
    .rangeBetween(
        start=Window.unboundedPreceding,
        end=0,
    )
)
w_month = (
    Window
    .partitionBy("date")
    .orderBy(F.desc("ca_item"))
)

df = (
    df
    .withColumn("ca_walmart", F.sum("ca").over(w_walmart))
    .withColumn("pct_ca_item", F.col("ca_item") / F.col("ca_walmart"))
    .withColumn("cum_pct_item", F.sum("pct_ca_item").over(w_month))

    .withColumn(
        "product_perf_category",
        F.when(F.col("cum_pct_item") <= 0.8, "A")
        .when(F.col("cum_pct_item") <= 0.95, "B")
        .otherwise("C")
    )
)

#4 - Lags statistiques agrégés
Les lags agrégés consistent à appliquer lags sur des agrégations (comme la somme, la moyenne, le minimum, le maximum) sur une fenêtre.

In [10]:
# Moyennes des ventes sur les 3 derniers mois pour chaque produit
w_ts = (
    Window
    .partitionBy("item_id", "store_id")
    .orderBy(F.asc(F.col("date")))
    .rowsBetween(
        start=-2,
        end=0,
    )
)
df = (
    df
    .withColumn("avg_ca_3months", F.avg("ca").over(w_ts))
)

In [11]:
# Lags des Moyennes des ventes sur les 3 derniers mois
ts_date_window = Window.partitionBy("item_id", "store_id").orderBy(F.asc("date"))
n_lags = 12
for i in range(1, n_lags + 1):
    df = (
        df
        .withColumn(f"avg_3month_lag{i}", F.lag("avg_ca_3months", i).over(ts_date_window))
    )

# Sauvegarde des résultats de calculs sur un fichier pour une série temporelle pour validation des résultats


In [ ]:
df.filter(F.col("id") == "FOODS_1_001_CA_1_validation").toPandas().to_csv("extract_2.csv", sep=";")